# Homework Assignment 1 - Maximilian Blesch, Aiko Schmeißer, Felix Degenhardt
## Exercise 8

#### First, install packages needed

In [134]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from scipy import stats

#### Read in the dataset:


In [135]:
df = pd.read_csv('A1Data2018.csv')
df

,Cost,output,Plabor,Pcapital,Pfuel
0,0.082,2,2.09,183,17.9
1,0.661,3,2.05,174,35.1
2,0.990,4,2.05,171,35.1
3,0.315,4,1.83,166,32.2
4,0.197,5,2.12,233,28.6
...,...,...,...,...,...
140,44.894,9956,1.68,203,28.8
141,67.120,11477,2.24,151,26.5
142,73.050,11796,2.12,148,28.6
143,139.422,14359,2.31,212,33.5


#### Take the logs as given in Hayashi:

In [136]:
df_log = df.apply(lambda x: np.log(x))
df_log

,Cost,output,Plabor,Pcapital,Pfuel
0,-2.501036,0.693147,0.737164,5.209486,2.884801
1,-0.414001,1.098612,0.717840,5.159055,3.558201
2,-0.010050,1.386294,0.717840,5.141664,3.558201
3,-1.155183,1.386294,0.604316,5.111988,3.471966
4,-1.624552,1.609438,0.751416,5.451038,3.353407
...,...,...,...,...,...
140,3.804304,9.205931,0.518794,5.313206,3.360375
141,4.206482,9.348100,0.806476,5.017280,3.277145
142,4.291144,9.375516,0.751416,4.997212,3.353407
143,4.937505,9.572132,0.837248,5.356586,3.511545


# a)

#### Estimate the model by OLS to obtain the following output:

In [137]:
est = sm.OLS(endog=df_log['Cost'], exog=sm.add_constant(df_log[df_log.columns[1:5]])).fit()
est.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   Cost   R-squared:                       0.926
Model:                            OLS   Adj. R-squared:                  0.924
Method:                 Least Squares   F-statistic:                     437.7
Date:                Sat, 14 Nov 2020   Prob (F-statistic):           4.82e-78
Time:                        11:24:48   Log-Likelihood:                -67.542
No. Observations:                 145   AIC:                             145.1
Df Residuals:                     140   BIC:                             160.0
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.5265      1.774     -1.987      0.049      -7.035      -0.018
output         0.7204      0.017     41.244      0.000       0.686       0.755
Plabor         0.4363      0.291      1.499      0.136      -0.139       1.012
Pcapital      -0.2199      0.339     -0.648      0.518      -0.891       0.451
Pfuel          0.4265      0.100      4.249      0.000       0.228       0.625
==============================================================================
Omnibus:                       51.403   Durbin-Watson:                   1.013
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              175.700
Skew:                           1.303   Prob(JB):                     7.03e-39
Kurtosis:                       7.721   Cond. No.                         506.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

#### As the regression output shows the model that has been estimated contains of five regressors: a constant and the log values of output, Plabor, Pcapital, Pfuel. As the dependent variable is the log Cost, the estimated coefficients show the partial ceteris paribus effect of a one percent change of each independent variable on the cost, on average. For example, a one percent increase in Pfuel is on average associated with a 0.4265 percent increase in cost, keeping all other factors constant. This is significant on the 1%-level of significance as the p-value is smaller than 0.01. Other coefficients can be interpreted correspondingly.

# b)
## a)

In [138]:
constraint1 = 'Pfuel = 0.4'

T_test = est.t_test(constraint1)
T_test

<class 'statsmodels.stats.contrast.ContrastResults'>
                             Test for Constraints                             
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
c0             0.4265      0.100      0.264      0.792       0.228       0.625

#### The simple t-test with H0: Pfuel = 0.4 yields the t-statistic 0.264 and the corresponding p-Value of 0.792. As the latter is clearly greater than 0.05 or every other reasonable level of significance, the null of Pfuel being equal to 0.4 cannot be rejected on the 5% level

## b)

Since we have 5 regressors and 1 restriction, the R- matrix has to be a 1x5 matrix and the r is a scalar that is equal to 0.4:

$R\beta = r$ $\rightarrow\begin{bmatrix}
0 & 0 & 0 & 0& 1\\
\end{bmatrix} \begin{bmatrix} const \\ output \\ Plabor \\ Pcapital \\ Pfuel \end{bmatrix} = 0.4$ 


#### define the matrices

In [139]:
R = np.matrix([0, 0., 0, 0, 1])
r = np.matrix([0.4])


In [156]:
def f_test_manually(est, R, r):
    cov = np.matrix(est.cov_params())
    b = np.matrix(est.params).transpose()
    f_stat = (np.matmul(R,b) - r).transpose() @ np.linalg.inv(R @ cov @ R.transpose()) @ (np.matmul(R,b) - r) / len(r)
    p_value = stats.f.sf(f_stat, len(r), (est.nobs - len(est.params)))
    return ('F-statistic: {}'.format(f_stat), 'F-p-value: {}'.format(p_value))

In [157]:
f_test_manually(est, R,r)

('F-statistic: [[0.06979861]]', 'F-p-value: [[0.79201874]]')

#### Verify the results by the implemented F-test:

In [142]:
F_test = est.f_test(constraint1)
F_test

<class 'statsmodels.stats.contrast.ContrastResults'>
<F test: F=array([[0.06979861]]), p=0.7920187421520621, df_denom=140, df_num=1>

#### The F-test with the specifications above shows the F-statistic of 0.0698 which has a p-Value of 0.79. This is again greater than 0.05 and the null can hence not be rejected on the 5% level of significance. This test can be considered correct as, for 1 restriction, the F-statistic is the squared t-statistic:

In [143]:
print(np.sqrt(F_test.fvalue))

[[0.26419427]]


## c)

#### As the unrestricted model has already been estimated, first regress the restricted model with the constraint: 

In [144]:
restr = sm.GLM(endog=df_log['Cost'], exog=sm.add_constant(df_log[df_log.columns[1:5]]))
est_restr  = restr.fit_constrained(constraint1)
est_restr.summary()

C:\Users\Felix\anaconda3\lib\site-packages\statsmodels\base\model.py:1362: RuntimeWarning: divide by zero encountered in true_divide
  return self.params / self.bse


<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                   Cost   No. Observations:                  145
Model:                            GLM   Df Residuals:                      141
Model Family:                Gaussian   Df Model:                            3
Link Function:               identity   Scale:                         0.15293
Method:                          IRLS   Log-Likelihood:                -67.578
Date:                Sat, 14 Nov 2020   Deviance:                       21.563
Time:                        11:24:48   Pearson chi2:                     21.6
No. Iterations:                     1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.5453      1.767     -2.006      0.045      -7.009      -0.082
output         0.7196      0.017     42.016      0.000       0.686       0.753
Plabor         0.4651      0.269      1.729      0.084      -0.062       0.992
Pcapital      -0.2024      0.332     -0.610      0.542      -0.853       0.448
Pfuel          0.4000          0        inf      0.000       0.400       0.400
==============================================================================

Model has been estimated subject to linear equality constraints.
"""

#### define a function that gives the F-statistic described in Hayashi as well as the corresponding p-Value:

In [145]:
def calculate_nested_f_statistic(restr, unrestr):
    """Given two fitted GLMs, the larger of which contains the parameter space of the smaller, return the F Stat and P value corresponding to the larger model adding explanatory power"""
    addtl_params = unrestr.df_model - restr.df_model
    f_stat = (restr.deviance - unrestr.ssr) / (addtl_params * unrestr.scale)
    df_numerator = addtl_params
    # use fitted values to obtain n_obs from model object:
    df_denom = (unrestr.fittedvalues.shape[0] - unrestr.df_model)
    p_value = stats.f.sf(f_stat, df_numerator, df_denom)
    return (f_stat, p_value)

In [146]:
print('F-Statistic:', calculate_nested_f_statistic(est_restr, est)[0], '\np-Value:', calculate_nested_f_statistic(est_restr, est)[1])

F-Statistic: 0.06979861222078099 
p-Value: 0.7920159893377923


Wald_test = ((est_restr.ssr - est.ssr)/(est.ssr / (est.nobs - len(est.params))))
Wald_test

#### As expected, the test-statistic and the p-value are the same for this type of testing. This can be further supported by checking the implemented Wald-test that leads the same values.

In [147]:
est.wald_test(constraint1)

<class 'statsmodels.stats.contrast.ContrastResults'>
<F test: F=array([[0.06979861]]), p=0.7920187421520621, df_denom=140, df_num=1>

## c)


In [148]:
#### First, estimate the unrestricted model again:

In [149]:
restr1 = sm.GLM(endog=df_log['Cost'], exog=sm.add_constant(df_log[df_log.columns[1:5]]))
est1 = restr1.fit()
est1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                   Cost   No. Observations:                  145
Model:                            GLM   Df Residuals:                      140
Model Family:                Gaussian   Df Model:                            4
Link Function:               identity   Scale:                         0.15394
Method:                          IRLS   Log-Likelihood:                -67.542
Date:                Sat, 14 Nov 2020   Deviance:                       21.552
Time:                        11:24:48   Pearson chi2:                     21.6
No. Iterations:                     3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -3.5265      1.774     -1.987      0.047      -7.004      -0.049
output         0.7204      0.017     41.244      0.000       0.686       0.755
Plabor         0.4363      0.291      1.499      0.134      -0.134       1.007
Pcapital      -0.2199      0.339     -0.648      0.517      -0.885       0.445
Pfuel          0.4265      0.100      4.249      0.000       0.230       0.623
==============================================================================
"""

#### Since we have 5 regressors and 1 restriction, the R- matrix has to be a 1x5 matrix and the r is a scalar that is equal to 1:

$R\beta = r$ $\rightarrow\begin{bmatrix}
0 & 0 & 1 & 1& 1\\
\end{bmatrix} \begin{bmatrix} const \\ output \\ Plabor \\ Pcapital \\ Pfuel \end{bmatrix} = 1$ 

#### This can be easily implemented as: 


In [150]:
constraint2 = 'Plabor + Pcapital + Pfuel = 1'

#### Again perform the F-Test as in Hayashi (1.4.9) first, with matrices defined above: 

In [159]:
R1 = np.matrix([0, 0., 1, 1, 1])
r1 = np.matrix([1])


In [160]:
f_test_manually(est1, R1, r1)

('F-statistic: [[0.57366039]]', 'F-p-value: [[0.45008093]]')

#### And again confirm the results by implemented F-test

In [161]:
Ftest2 = est1.f_test(constraint2)
Ftest2

<class 'statsmodels.stats.contrast.ContrastResults'>
<F test: F=array([[0.57366039]]), p=0.4500809321271022, df_denom=140, df_num=1>

#### The null of homogeneity can hence not be rejected on the 5%- level of significance as the p-value is clearly greater than 0.05.


#### For the other test approach (Wald), the restricted model has to be estimated again:


In [152]:
est2 = restr1.fit_constrained(constraint2)
est2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                   Cost   No. Observations:                  145
Model:                            GLM   Df Residuals:                      141
Model Family:                Gaussian   Df Model:                            3
Link Function:               identity   Scale:                         0.15348
Method:                          IRLS   Log-Likelihood:                -67.838
Date:                Sat, 14 Nov 2020   Deviance:                       21.640
Time:                        11:24:48   Pearson chi2:                     21.6
No. Iterations:                     1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -4.6908      0.885     -5.301      0.000      -6.425      -2.956
output         0.7207      0.017     41.334      0.000       0.687       0.755
Plabor         0.5929      0.205      2.898      0.004       0.192       0.994
Pcapital      -0.0074      0.191     -0.039      0.969      -0.381       0.366
Pfuel          0.4145      0.099      4.189      0.000       0.221       0.608
==============================================================================

Model has been estimated subject to linear equality constraints.
"""

#### Where R and r stated above can be confirmed (second and third element in the list)

#### Again state the function for calculating the Wald-test statistic

In [153]:
def calculate_nested_f_statistic1(restr, unrestr):
    """Given two fitted GLMs, the larger of which contains the parameter space of the smaller, return the F Stat and P value corresponding to the larger model adding explanatory power"""
    addtl_params = unrestr.df_model - restr.df_model
    f_stat = (restr.deviance - unrestr.deviance) / (addtl_params * unrestr.scale)
    df_numerator = addtl_params
    # use fitted values to obtain n_obs from model object:
    df_denom = (unrestr.fittedvalues.shape[0] - unrestr.df_model)
    p_value = stats.f.sf(f_stat, df_numerator, df_denom)
    return (f_stat, p_value)

In [154]:
print('F-Statistic:', calculate_nested_f_statistic1(est2, est1)[0], '\np-Value:', calculate_nested_f_statistic1(est2, est1)[1])

F-Statistic: 0.573660387505294 
p-Value: 0.4500719174519835


In [155]:
est.wald_test(constraint2)

<class 'statsmodels.stats.contrast.ContrastResults'>
<F test: F=array([[0.57366039]]), p=0.4500809321271022, df_denom=140, df_num=1>

#### The test statistic and corresponding p-Values of the F-test, the manually calculated Wald-test and the Wald-test provided by the software are the same with the test decision stated above. 